In [ ]:
# Install the libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

In [2]:
# Import the libraries
import torch
import gc

import pandas as pd
import numpy as np

from datasets import Dataset
from datasets import load_metric
from datasets import load_dataset

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

from huggingface_hub import notebook_login

In [4]:
# Check if GPU is available
torch.cuda.is_available()

True

In [5]:
# Clear the cuda cache
gc.collect()
torch.cuda.empty_cache()

In [30]:
# Log into the hugging face account
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [6]:
# Load data set from the hugging face account
dataset = load_dataset("MokaExpress/Unil")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/MokaExpress___csv/MokaExpress--Unil-1ec37ae8a2bb4385/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
# Have a look at the data set
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence', 'difficulty'],
        num_rows: 4800
    })
})


In [8]:
# Remove the column "id"
dataset = dataset.remove_columns("id")

In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'difficulty'],
        num_rows: 4800
    })
})


In [10]:
# Rename the column "difficulty" to "label" for the hugging face model predictions 
dataset = dataset.rename_column("difficulty", "label")

In [11]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 4800
    })
})


In [12]:
# Note the difficulty levels are mapped to integers during a preprocessing
dataset["train"][0]

{'sentence': "Les coûts kilométriques réels peuvent diverger sensiblement des valeurs moyennes en fonction du moyen de transport utilisé, du taux d'occupation ou du taux de remplissage, de l'infrastructure utilisée, de la topographie des lignes, du flux de trafic, etc.",
 'label': 4}

In [13]:
# Split train test data set, ratio 9:1
train_test_dataset = dataset["train"].train_test_split(test_size=0.1)

In [14]:
# Check if the split is done correctly
train_test_dataset["test"]

Dataset({
    features: ['sentence', 'label'],
    num_rows: 480
})

In [19]:
# Read the unlabelled test data 
df = pd.read_csv("unlabelled_test_data.csv")

In [20]:
# Convert the pandas data frame to the hugging face data set
ds = Dataset.from_pandas(df)

In [21]:
# Remove the column "id"
ds = ds.remove_columns("id")

In [22]:
ds

Dataset({
    features: ['sentence'],
    num_rows: 1200
})

In [29]:
# Define the evaluation
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   return {"accuracy": accuracy}

In [44]:
# Try out different pre trained hugging face models to compare the results

#tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
#model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=6)

#tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-en-fr-cased")
#model = AutoModelForSequenceClassification.from_pretrained("Geotrend/distilbert-base-en-fr-cased", num_labels=6)

tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-en-fr-cased")
model = AutoModelForSequenceClassification.from_pretrained("Geotrend/bert-base-en-fr-cased", num_labels=6)

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/659 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-en-fr-cased/snapshots/30b1dd5115bb2441a8c098ff08aa67048e70c71d/config.json
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-en-fr-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.25.1",
  "type_vocab_size":

Downloading:   0%|          | 0.00/230k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-en-fr-cased/snapshots/30b1dd5115bb2441a8c098ff08aa67048e70c71d/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-en-fr-cased/snapshots/30b1dd5115bb2441a8c098ff08aa67048e70c71d/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-en-fr-cased/snapshots/30b1dd5115bb2441a8c098ff08aa67048e70c71d/config.json
Model config BertConfig {
  "_name_or_path": "Geotrend/bert-base-en-fr-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_pro

Downloading:   0%|          | 0.00/447M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--Geotrend--bert-base-en-fr-cased/snapshots/30b1dd5115bb2441a8c098ff08aa67048e70c71d/pytorch_model.bin
Some weights of the model checkpoint at Geotrend/bert-base-en-fr-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a 

In [45]:
# Tokenize the input
def preprocess_function(examples):
   return tokenizer(examples["sentence"], truncation=True)

In [46]:
# Tokenize train test data sets
tokenized_train = train_test_dataset["train"].map(preprocess_function, batched=True)
tokenized_test = train_test_dataset["test"].map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [47]:
# Sample a tokenized sentence to have a look
tokenized_test[56]

{'sentence': "Cette activité apparait dans la religion grecque de l'Antiquité comme un cadeau qui est toutefois imposé aux Hommes pour qu'ils puissent survivre",
 'label': 5,
 'input_ids': [11,
  2219,
  8571,
  24062,
  1886,
  536,
  367,
  232,
  5818,
  26314,
  227,
  88,
  20,
  8312,
  28383,
  845,
  242,
  943,
  5589,
  259,
  444,
  297,
  13468,
  325,
  16712,
  702,
  13235,
  230,
  419,
  607,
  20,
  2157,
  27384,
  452,
  423,
  32618,
  354,
  12],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

In [48]:
# Tokenize the unlabelled test data
tokenized_ds = ds.map(preprocess_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [49]:
# Sample a tokenized sentence to have a look
tokenized_ds[0]

{'sentence': 'Nous dûmes nous excuser des propos que nous eûmes prononcés',
 'input_ids': [11,
  17756,
  5920,
  2360,
  4794,
  1141,
  2798,
  286,
  260,
  17044,
  244,
  4794,
  81,
  23622,
  2360,
  1043,
  4381,
  14583,
  12],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [50]:
# Set the data collector using the tokenizer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [51]:
# Define the training arguments and a trainer with all constructed objects
repo_name = "french-difficulty_detection_models_hugging_face"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)
 
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/content/french-difficulty_detection_models_hugging_face is already a clone of https://huggingface.co/MokaExpress/french-difficulty_detection_models_hugging_face. Make sure you pull the latest changes with `repo.git_pull()`.


In [52]:
# Fine tune the model using the specified train test data sets
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4320
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 540
  Number of trainable parameters = 111702534
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using 

Step,Training Loss
500,1.108000


Saving model checkpoint to french-difficulty_detection_models_hugging_face/checkpoint-270
Configuration saved in french-difficulty_detection_models_hugging_face/checkpoint-270/config.json
Model weights saved in french-difficulty_detection_models_hugging_face/checkpoint-270/pytorch_model.bin
tokenizer config file saved in french-difficulty_detection_models_hugging_face/checkpoint-270/tokenizer_config.json
Special tokens file saved in french-difficulty_detection_models_hugging_face/checkpoint-270/special_tokens_map.json
tokenizer config file saved in french-difficulty_detection_models_hugging_face/tokenizer_config.json
Special tokens file saved in french-difficulty_detection_models_hugging_face/special_tokens_map.json
Several commits (2) will be pushed upstream.
Saving model checkpoint to french-difficulty_detection_models_hugging_face/checkpoint-540
Configuration saved in french-difficulty_detection_models_hugging_face/checkpoint-540/config.json
Model weights saved in french-difficulty_

TrainOutput(global_step=540, training_loss=1.0944329509028683, metrics={'train_runtime': 176.0946, 'train_samples_per_second': 49.065, 'train_steps_per_second': 3.067, 'total_flos': 423286889632704.0, 'train_loss': 1.0944329509028683, 'epoch': 2.0})

In [53]:
# Evalute the results
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 480
  Batch size = 16


{'eval_loss': 1.0792219638824463,
 'eval_accuracy': 0.5479166666666667,
 'eval_runtime': 2.8095,
 'eval_samples_per_second': 170.847,
 'eval_steps_per_second': 10.678,
 'epoch': 2.0}

In [68]:
# Use the fine tuned model to predict
pred = trainer.predict(tokenized_ds)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence. If sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1200
  Batch size = 16


In [69]:
# Check if the length of the predictions is correct
print(pred.predictions.shape)

(1200, 6)


In [70]:
# Convert the predcition to labels
pred_kaggle = np.argmax(pred.predictions, axis=-1)

In [71]:
# Check if the results are converted correctly
pred_kaggle

array([5, 1, 2, ..., 5, 3, 4])

In [72]:
# Read the submission format
df_pred = pd.read_csv("sample_submission.csv")
# Replace the dummy results with predictions
df_pred.difficulty = pred_kaggle

In [73]:
# Map integers labels back to the language difficulties
df_pred.difficulty = df_pred.difficulty.map({0:'A1', 1:'A2', 2:'B1', 3:'B2', 4:'C1', 5:'C2'})
df_pred

,id,difficulty
0,0,C2
1,1,A2
2,2,B1
3,3,A2
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A2
1197,1197,C2
1198,1198,B2


In [74]:
# Write the results to a csv file for submisstion on Kaggle
df_pred.to_csv("submission_22_12_geotrend_bert-base-en-fr-cased.csv", index=False)